In [1]:
# -*- coding: utf-8 -*-
import math, random
import time

## Parámetros del sistema

In [2]:
side = 50
size = side*side
ntrials = side**4 # Es buena idea que vaya como n*m
nsamples = size
j_ising = -1.0
t_env = 2.0

## Funciones 

In [3]:
def change_spin():
    return random.choice([-1, 1])

In [4]:
def initialize(side):
    spins = []
    neighbors = []
    for jj in range(side):
        for ii in range(side):
            index = jj*side + ii
            spins.append(change_spin())
            all_nbs = set([((jj + col)%side)*side + (ii + row)%side 
                for col in range(-1, 2) for row in range(-1, 2)])
            diag_nbs = set([((jj + col)%side)*side + (ii + row)%side
                for col in [-1, 1] for row in [-1, 1]])
            neighbors.append(list(all_nbs - diag_nbs - {index}))
    return spins, neighbors

In [5]:
def montecarlo_cycle(size, spins, neighbors, ntrials, nsamples, j_ising, t_env):
    nerrors = 0
    energies = []
    magnets = []
    
    te = 0.0
    for jj in range(size):
        nbs = neighbors[jj]
        neighbors_spins = [spins[nb] for nb in nbs]
        #neighbors_spins = spins[neighbors[jj]] #Lista de los espines vecinos
        te = te + potential_energy(spins[jj], neighbors_spins, j_ising) 
    te = te/2.0
    energies.append(te)
    magnets.append(magnetization(spins))
    
    for ii in range(ntrials):
        spins, energy_change = montecarlo(size, spins, neighbors, j_ising, t_env)
        if ii%nsamples == 0:
            energies.append(te)
            magnets.append(magnetization(spins))
    
    return spins, energies, magnets, nerrors

In [6]:
def montecarlo(size, spins, neighbors, j_ising, t_env):
    ii = random.randint(0, size-1)
    nbs = neighbors[ii] #Es mejor así que introduciéndolo directamente a la comprensión
    neighbors_spins = [spins[nb] for nb in nbs]

    old_energy = potential_energy(spins[ii], neighbors_spins, j_ising)
    trial_spin = -spins[ii]
    #trial_energy = potential_energy(trial_spin, neighbors_spins, j_ising)
    #energy_diff = trial_energy - old_energy
    energy_diff = -2.0*old_energy
    energy_change = 0.0
    if metropolis_step(energy_diff/t_env):
        spins[ii] = trial_spin
        energy_change = energy_diff
    return spins, energy_change

In [7]:
def potential_energy(central_spin, neighbors_spins, j_ising):
    return j_ising*central_spin*sum(neighbors_spins)

In [8]:
def magnetization(spins):
    return sum(spins)

In [9]:
def metropolis_step(exp_arg):
    """
    Returns True if the trial configuration goes towards a region with higher probability
    or it is given the chance to explore regions with fewer probability
    """
    if exp_arg < 0.0: # Energía: w < 0
        return True # Se actualiza el estado del sistema
    else:
        w = math.exp(-exp_arg)
        if random.random() < w: # Energía: w
            return True # También se actualiza el estado del sistema
        else:
            return False

## Programa principal

In [10]:
spins, neighbors = initialize(side)

In [11]:
#spins, energy_change = montecarlo(size, spins, neighbors, j_ising, t_env)

In [12]:
t1 = time.time()
spins, energies, magnets, nerrors = montecarlo_cycle(size, spins, neighbors, ntrials, nsamples, j_ising, t_env)
t2 = time.time()
print(t2-t1)

39.17380952835083


In [13]:
#1.0985674858093262 s con 20x20 
#45.51250720024109 s con 50x50
#44.53565788269043 s 50x50 usando -2*old_energy sin tratar de optimizar con las listas de posibles valores de energías